[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ncerdan/HandsOnML/blob/master/Ch_08_Dimensionality_Reduction.ipynb)

# PCA

## Principal Components

In [1]:
# make a 3D dataset
import numpy as np

np.random.seed(4)
m = 60
w1, w2 = 0.1, 0.3
noise = 0.1

angles = np.random.rand(m) * 3 * np.pi / 2 - 0.5
X = np.empty((m, 3))
X[:, 0] = np.cos(angles) + np.sin(angles)/2 + noise * np.random.randn(m) / 2
X[:, 1] = np.sin(angles) * 0.7 + noise * np.random.randn(m) / 2
X[:, 2] = X[:, 0] * w1 + X[:, 1] * w2 + noise * np.random.randn(m)

In [6]:
# you can compute them by hand using svd
# if computing by hand, be sure to center the dataset!

# compute svd
X_centered = X - X.mean(axis=0)
U, s, vT = np.linalg.svd(X_centered)

# extract the first two PCs
c1 = vT.T[:, 0]
c2 = vT.T[:, 1]

## Projecting Down into d Dimensions

In [7]:
# to project into d-dimensions, just use the first d PCs
# here: w=2

W_2 = vT.T[:, :2]
X_2D = X_centered.dot(W_2)

## Using Sklearn

In [8]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)   # projects to 2-D
X_2D = pca.fit_transform(X)

# the components_ attr holds the W_d matrix used
print('Components:', pca.components_)

# this would be the first PC
print('First PC:', pca.components_.T[:, 0])

Components: [[-0.93636116 -0.29854881 -0.18465208]
 [ 0.34027485 -0.90119108 -0.2684542 ]]
First PC: [-0.93636116 -0.29854881 -0.18465208]


## Explained Variance Ratio

In [9]:
# shows how much variance each PC contains
pca.explained_variance_ratio_   # as you can see, the third PC only would have 2%

array([0.84248607, 0.14631839])

## Choosing the Right Number of Dimensions

In [10]:
# first lets load in MNIST
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1)
mnist_target = mnist.target.astype(np.uint8)

In [11]:
# and partition it
from sklearn.model_selection import train_test_split

X = mnist['data']
y = mnist['target']

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [12]:
# here, we can use pca without reducing the data then pick the smallest
# dimension that maintains at least 95% of the variance

pca = PCA()
pca.fit(X_train)
cumulativeSum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumulativeSum >= 0.95) + 1
print(d)

154


In [13]:
# then we could run
pca = PCA(n_components=d)
X_reduced = pca.fit_transform(X_train)

In [14]:
# however, instead of this, this is built in to the PCA class
# if you pass a float between 0 and 1 it will assume you are 
# setting the minimum variance retained:

pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X_train)

## PCA for Compression

In [15]:
# this code 'compresses' the data, but then also reconstructs it
pca = PCA(n_components=154)
X_reduced = pca.fit_transform(X_train)
X_recovered = pca.inverse_transform(X_reduced)

## Randomized PCA

In [16]:
# the svd_solver='randomized' uses a stochastic PCA algo
#   the default value is 'auto' where sklearn uses the following values:
#       'randomized': if m or n is greater than 500 and d is less than 80% of m or n
#       'full' approach: otherwise

rnd_pca = PCA(n_components=154, svd_solver='randomized')
X_reduced = rnd_pca.fit_transform(X_train)

## Incremental PCA

In [17]:
from sklearn.decomposition import IncrementalPCA

# does mini-batches of size 100
n_batches = 100
inc_pca = IncrementalPCA(n_components=154)
for X_batch in np.array_split(X_train, n_batches):
    inc_pca.partial_fit(X_batch)

X_reduced = inc_pca.transform(X_train)

In [18]:
# you could also use numpy's memmap class which allows you to operate on binary data 
# on disk as if it is all in memory

# X_mm = np.memmap(filename, dtype='float32', mode='readonly', shape=(m, n))

# batch_size = m // n_batches
# inc_pca = IncrementalPCA(n_components=154, batch_size=batch_size)
# inc_pca.fit(X_mm)

# Kernal PCA

In [12]:
# get some dummy data
from sklearn.datasets import make_swiss_roll
X, t = make_swiss_roll(n_samples=1000, noise=0.2, random_state=42)
y = t > 6.9

In [13]:
# would use sklearn's KernelPCA class imported as follows
from sklearn.decomposition import KernelPCA

## Selecting a Kernel and Tuning Hyperparameters

In [14]:
# creates a simple 2-step pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

clf = Pipeline([
    ('kpca', KernelPCA(n_components=2)),
    ('log_reg', LogisticRegression())
])

param_grid = [{
    'kpca__gamma': np.linspace(0.03, 0.05, 5),
    'kpca__kernel': ['rbf', 'sigmoid']
}]

grid_search = GridSearchCV(clf, param_grid, cv=3)
grid_search.fit(X, y)

GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('kpca',
                                        KernelPCA(alpha=1.0, coef0=1,
                                                  copy_X=True, degree=3,
                                                  eigen_solver='auto',
                                                  fit_inverse_transform=False,
                                                  gamma=None, kernel='linear',
                                                  kernel_params=None,
                                                  max_iter=None, n_components=2,
                                                  n_jobs=None,
                                                  random_state=None,
                                                  remove_zero_eig=False,
                                                  tol=0)),
                                       ('log_reg',
                                 

In [17]:
# can also create the pre-image
rbf_pca = KernelPCA(n_components=2, kernel='rbf', gamma=0.0433, fit_inverse_transform=True) # last param allows for inverse_transform

X_reduced = rbf_pca.fit_transform(X)
X_preimage = rbf_pca.inverse_transform(X_reduced)

In [19]:
# then you can compute the pre-image error
from sklearn.metrics import mean_squared_error
mean_squared_error(X, X_preimage)       # would use this to gridsearch

32.78630879576616

# Exercises

## 9.) Load MNIST, and compare training time before and after dimensionality reduction

In [20]:
# get the data
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1)
X = mnist['data']
y = mnist['target']

In [22]:
# split the data into training and testing
X_train = X[:60000]
X_test = X[60000:]
y_train = y[:60000]
y_test = y[60000:]

In [27]:
# make a model
from sklearn.ensemble import RandomForestClassifier

rnd_fst_clf = RandomForestClassifier(n_estimators=50)

In [28]:
# time its training
import time

start_t = time.time()
rnd_fst_clf.fit(X_train, y_train)
end_t = time.time()

print('time: {:.2f}s'.format(end_t - start_t))

time: 24.96s


In [29]:
# get the reduced dataset
from sklearn.decomposition import PCA

pca = PCA(n_components=0.95)
X_train_reduced = pca.fit_transform(X_train)

In [30]:
# time the reduced training

rnd_fst_clf2 = RandomForestClassifier(n_estimators=50)

start_t_r = time.time()
rnd_fst_clf2.fit(X_train_reduced, y_train)
end_t_r = time.time()

print('time: {:.2f}s'.format(end_t_r - start_t_r))  # on different classifier it would go faster --> not on random forests apparently

time: 59.14s


In [31]:
# now lets check out their performances
from sklearn.metrics import accuracy_score

X_test_reduced = pca.transform(X_test)

y_pred_normal = rnd_fst_clf.predict(X_test)
y_pred_redux  = rnd_fst_clf2.predict(X_test_reduced)

print('Normal: ', accuracy_score(y_test, y_pred_normal))
print('Reduced: ', accuracy_score(y_test, y_pred_redux))

Normal:  0.9669
Reduced:  0.9457


In [32]:
# as you can see, reducing the dimension slightly lowered the performance (as expected)